In [1]:
# 최종수정 2024/06/04 12:32
# 플라스틱 컵 분류기, 이진 분류 모델
# ./Data_PlasticCup/clean ./Data_PlasticCup/ng 디렉토리의 사진을 바탕으로 학습

import os
import cv2
from sklearn.model_selection import train_test_split
data = []
labels = []

# Clean 폴더 이미지 불러오기
clean_dir = 'Data_PlasticCup/clean'
for filename in os.listdir(clean_dir):
    img = cv2.imread(os.path.join(clean_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(0)  # 0 = 깨끗한 컵

# NG 폴더 이미지 불러오기
ng_dir = 'Data_PlasticCup/ng'
for filename in os.listdir(ng_dir):
    img = cv2.imread(os.path.join(ng_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(1)  # 1 = 더러운 컵

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import accuracy_score

# 데이터를 NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# 모델 정의
model = Sequential()

# 첫 번째 컨볼루션 레이어
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3))) # 수정된 부분
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 2 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 3
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))

# fully connected 레이어
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30, batch_size=16)

# 모델 저장
model.save('plastic_cup_classifier.h5')

Epoch 1/30


c:\Users\lavie\.conda\envs\ENV00\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.5743 - loss: 72.2921 - val_accuracy: 0.7115 - val_loss: 2.9160
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8059 - loss: 0.9911 - val_accuracy: 0.9615 - val_loss: 0.1350
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.8701 - loss: 0.2442 - val_accuracy: 0.9808 - val_loss: 0.0534
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9437 - loss: 0.1114 - val_accuracy: 0.9808 - val_loss: 0.0497
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9748 - loss: 0.1069 - val_accuracy: 0.9808 - val_loss: 0.0286
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9844 - loss: 0.0433 - val_accuracy: 0.9808 - val_loss: 0.0360
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9901 - loss: 0.0306 - val_accuracy: 1.0000 - val_loss: 0.0195
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9785 - loss: 0.0599 - val_accuracy: 1.0000 - val_loss: 0

In [3]:
from tensorflow.keras.models import load_model

# 모델 불러오기
model = load_model('plastic_cup_classifier.h5')

# 테스트 데이터셋 준비
x_test = np.array(x_val)
y_test = np.array(y_val)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# 예측값 구하기
y_pred = (model.predict(x_test) > 0.5).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (using sklearn.metrics): {accuracy * 100:.2f}%')

2/2 - 0s - 99ms/step - accuracy: 1.0000 - loss: 0.0375
Test accuracy: 100.00%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Accuracy (using sklearn.metrics): 100.00%
